In [1]:
import pandas as pd
import numpy as np
import os
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams['font.sans-serif'] = "Arial"

In [3]:
TimePoint_list = ['0hpa', '6hpa', '12hpa', '1dpa', '3dpa', '7dpa']
CellType_list = ['Epidermal', 'Gut', 'Muscle', 'Neoblast', 'Neuronal', 'Parenchymal', 'Secretory']

In [4]:
adata_list = {}
for temp_timepoint in TimePoint_list:
    adata_list[temp_timepoint] = sc.read('../../Data/Visium/Visium_'+temp_timepoint+'.h5ad')
    adata_list[temp_timepoint].obs['Timepoint'] = temp_timepoint

# Wound-A

In [5]:
for temp_timepoint in TimePoint_list:
    print(temp_timepoint, adata_list[temp_timepoint].shape)
    adata_list[temp_timepoint] = adata_list[temp_timepoint][adata_list[temp_timepoint].obs['AP_Info'].isin(['AP-1', 'AP-2'])]
    print(adata_list[temp_timepoint].shape)

0hpa (8949, 27664)
(1319, 27664)
6hpa (7304, 27664)
(235, 27664)
12hpa (10156, 27664)
(759, 27664)
1dpa (8423, 27664)
(460, 27664)
3dpa (8365, 27664)
(366, 27664)
7dpa (10210, 27664)
(485, 27664)


In [18]:
Cell2location_df = {}
Cell2location_rate_df = {}
for temp_timepoint in TimePoint_list:
    Cell2location_df[temp_timepoint] = adata_list[temp_timepoint].obs.loc[:, ['cell2location_'+x for x in CellType_list]]
    Cell2location_df[temp_timepoint].columns = [x.split('_')[1] for x in Cell2location_df[temp_timepoint].columns]
    Cell2location_rate_df[temp_timepoint] = (Cell2location_df[temp_timepoint].T/np.sum(Cell2location_df[temp_timepoint].T)).T

In [19]:
co_list = {}
for temp_timepoint in TimePoint_list:
    print(temp_timepoint)
    co_list[temp_timepoint] = {}
    for it in Cell2location_rate_df[temp_timepoint].index:
        temp_df = Cell2location_rate_df[temp_timepoint].loc[it, Cell2location_rate_df[temp_timepoint].loc[it,:]>0.1]
        temp_df = temp_df.sort_values()[::-1]
        co_list[temp_timepoint][it] = np.array(temp_df.index[:3])

0hpa
6hpa
12hpa
1dpa
3dpa
7dpa


In [22]:
co_df = {}
for temp_timepoint in TimePoint_list:
    co_df[temp_timepoint] = pd.DataFrame(0, index=CellType_list, columns=CellType_list)
    print(temp_timepoint)
    for it in co_list[temp_timepoint].keys():
        temp_list = co_list[temp_timepoint][it]
        if temp_list.shape[0]==1:
            continue
        for it1 in temp_list:
            for it2 in temp_list:
                if it1==it2:
                    continue
                co_df[temp_timepoint].loc[it1, it2] += 1

0hpa
6hpa
12hpa
1dpa
3dpa
7dpa


In [24]:
for temp_tp in TimePoint_list:
    co_df[temp_tp].to_csv('Colocation/'+temp_tp+'_Wound-A.txt', sep='\t')